In [1]:
from mlscorecheck.check import (check_kfold_rom_scores)
from mlscorecheck.core import (IntervalUnion)
from mlscorecheck.utils import (calculate_scores,
                                generate_problems_with_folds)

In [2]:
k = 4
eps = 10**(-k)

In [3]:
folds, problem = generate_problems_with_folds(n_repeats=2, 
                                                n_folds=3,
                                                random_seed=5)
problem, folds

({'p': 869, 'n': 208, 'n_folds': 3, 'n_repeats': 2},
 [{'p': 290, 'n': 69, 'tp': 190, 'tn': 17},
  {'p': 290, 'n': 69, 'tp': 74, 'tn': 9},
  {'p': 289, 'n': 70, 'tp': 229, 'tn': 63},
  {'p': 290, 'n': 69, 'tp': 113, 'tn': 31},
  {'p': 290, 'n': 69, 'tp': 209, 'tn': 8},
  {'p': 289, 'n': 70, 'tp': 205, 'tn': 16}])

In [4]:
scores = calculate_scores(folds, strategy='rom', rounding_decimals=k)
scores

{'acc': 0.5404,
 'sens': 0.5869,
 'spec': 0.3462,
 'npv': 0.1671,
 'ppv': 0.7895,
 'bacc': 0.4665,
 'f1p': 0.6733,
 'fm': 0.6807}

In [5]:
flag, details = check_kfold_rom_scores(scores, 
                                        eps=eps, 
                                        dataset=problem, 
                                        return_details=True)

2023-08-17 11:10:02,093:INFO:checking acc and sens against spec
2023-08-17 11:10:02,094:INFO:checking acc and sens against npv
2023-08-17 11:10:02,096:INFO:checking acc and sens against ppv
2023-08-17 11:10:02,097:INFO:checking acc and sens against bacc
2023-08-17 11:10:02,098:INFO:checking acc and sens against f1p
2023-08-17 11:10:02,099:INFO:checking acc and sens against fm
2023-08-17 11:10:02,100:INFO:checking acc and spec against sens
2023-08-17 11:10:02,101:INFO:checking acc and spec against npv
2023-08-17 11:10:02,101:INFO:checking acc and spec against ppv
2023-08-17 11:10:02,102:INFO:checking acc and spec against bacc
2023-08-17 11:10:02,104:INFO:checking acc and spec against f1p
2023-08-17 11:10:02,105:INFO:checking acc and spec against fm
2023-08-17 11:10:02,105:INFO:checking acc and npv against sens
2023-08-17 11:10:02,106:INFO:checking acc and npv against spec
2023-08-17 11:10:02,107:INFO:checking acc and npv against ppv
2023-08-17 11:10:02,108:INFO:checking acc and npv agai

In [6]:
assert flag

In [7]:
for tmp in details['succeeded']:
    assert tmp['consistency']
    for item in tmp['details']:
        if item.get('target_interval_reconstructed') is not None and item.get('consistency'):
            target_interval = item['target_interval_reconstructed']
            iun = IntervalUnion(target_interval)
            assert not iun.intersection(IntervalUnion(item['target_interval'])).is_empty()

In [8]:
scores['bacc'] = 0.9

In [9]:
flag, details = check_kfold_rom_scores(scores, 
                                        eps=eps, 
                                        dataset=problem, 
                                        return_details=True)

2023-08-17 11:10:02,344:INFO:checking acc and sens against spec
2023-08-17 11:10:02,345:INFO:checking acc and sens against npv
2023-08-17 11:10:02,346:INFO:checking acc and sens against ppv
2023-08-17 11:10:02,347:INFO:checking acc and sens against bacc
2023-08-17 11:10:02,349:INFO:checking acc and sens against f1p
2023-08-17 11:10:02,350:INFO:checking acc and sens against fm
2023-08-17 11:10:02,351:INFO:checking acc and spec against sens
2023-08-17 11:10:02,352:INFO:checking acc and spec against npv
2023-08-17 11:10:02,353:INFO:checking acc and spec against ppv
2023-08-17 11:10:02,354:INFO:checking acc and spec against bacc
2023-08-17 11:10:02,355:INFO:checking acc and spec against f1p
2023-08-17 11:10:02,356:INFO:checking acc and spec against fm
2023-08-17 11:10:02,358:INFO:checking acc and npv against sens
2023-08-17 11:10:02,359:INFO:checking acc and npv against spec
2023-08-17 11:10:02,360:INFO:checking acc and npv against ppv
2023-08-17 11:10:02,361:INFO:checking acc and npv agai

In [10]:
assert not flag